Functions and imports


In [1]:
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), "..", "..", ".."))
main_copy_dir = os.path.join(parent_dir, "main_copy")
octagon_analysis = os.path.join(parent_dir, "octagon_analysis")
sys.path.append(main_copy_dir)
sys.path.append(octagon_analysis)

print("Added paths to sys.path:", main_copy_dir, octagon_analysis)

Added paths to sys.path: /Users/benny/Desktop/MSc/Project/Git/repos/main_copy /Users/benny/Desktop/MSc/Project/Git/repos/octagon_analysis


Imports with autoreload

In [3]:
%load_ext autoreload
%autoreload 2

import analysis.trajectory_efficiency as trajectory_efficiency
import analysis.opponent_visibility as opponent_visibility
import pandas as pd
import numpy as np
import parse_data.prepare_data as prepare_data
import parse_data.identify_filepaths as identify_filepaths
import analysis.conditioned_player_choice as conditioned_player_choice
import prepare_questionnaire_data
import data_extraction.get_indices as get_indices
import globals

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Define data folder and eprsonal data file

In [4]:
data_folder = '/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/questionnaire_analysis/pseudonymised_json_files'
personal_data_csv = '/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/questionnaire_analysis/questionnaire_data/personal_data.csv'

Prepare questionnaire data

In [5]:
questionnaire_data_df = prepare_questionnaire_data.preprocess_data()

AQ_scores.csv
GAD_scores.csv
BIS_scores.csv


/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/questionnaire_analysis/questionnaire_data/prepare_questionnaire_data.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = df[column_name].replace(answers_dict[trait]).astype(int)
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/questionnaire_analysis/questionnaire_data/prepare_questionnaire_data.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = df[column_name].replace(answers_dict[trait]).astype(int)
/Users/benny

Get social and solo filenames (solos ordered by pseudonym, neither ordered by date)

In [7]:
social_files, ordered_solos = identify_filepaths.get_filenames(data_folder=data_folder)

Create list of tuples for pseudonyms in socials

In [8]:
import re
pseudo_tuples_list = []
experiment_dates_socials = []
for file in social_files:
     # match the session number and the pseudonym string
    match = re.search(r'(\d+_\d)[/\\](\d{4}-\d{2}-\d{2})_(\d{2}-\d{2}-\d{2})_(.*?_.*?)_Social\.json', file)
    if match:
        session, date, time, pseudonyms = match.groups()
        pseudo_tuples = pseudonyms.split('_')[0::]
        pseudo_tuples_list.append(pseudo_tuples)
        experiment_dates_socials.append(date)

In [9]:
social_dates = {}
opponent_dict = {}

for i, pseudonym in enumerate(pseudo_tuples_list):
    player0 = pseudonym[0]
    player1 = pseudonym[1]

    if player0 not in social_dates:
        social_dates[player0] = []
        opponent_dict[player0] = []
    if player1 not in social_dates:
        social_dates[player1] = []
        opponent_dict[player1] = []

    social_dates[player0] = experiment_dates_socials[i]
    social_dates[player1] = experiment_dates_socials[i]

    opponent_dict[player0] = player1
    opponent_dict[player1] = player0


Prepare data and get trial lists

In [10]:
df_socials, social_lists = prepare_data.prepare_data(data_folder, social_files, combine=False)

filepath: /Users/benny/Desktop/MSc/Project/Git/repos/main_copy/questionnaire_analysis/pseudonymised_json_files/250417_1/2025-04-17_12-22-26_CY17_RL17_Social.json
Loading complete.
Preprocessing complete.
filepath: /Users/benny/Desktop/MSc/Project/Git/repos/main_copy/questionnaire_analysis/pseudonymised_json_files/241112_2/2024-11-12_15-23-24_FA12_SL12_Social.json
Loading complete.
Preprocessing complete.
filepath: /Users/benny/Desktop/MSc/Project/Git/repos/main_copy/questionnaire_analysis/pseudonymised_json_files/241210_1/2024-12-10_14-21-17_TE10_TK10_Social.json
Loading complete.
Preprocessing complete.
filepath: /Users/benny/Desktop/MSc/Project/Git/repos/main_copy/questionnaire_analysis/pseudonymised_json_files/250509_1/2025-05-09_12-30-13_AM09_MC09_Social.json
Loading complete.
Preprocessing complete.
filepath: /Users/benny/Desktop/MSc/Project/Git/repos/main_copy/questionnaire_analysis/pseudonymised_json_files/250401_2/2025-04-01_15-29-49_AL01_NL01_Social.json
Loading complete.
Prep

Choice conditioned on visibility, using conditioned_player_choice.py

Solo-social difference needs matching by pseudonym first

P(H|H first vis)
- first solo 
- second solo 
- combined solo 
- social 
- combined solo - social
- wall sep 1 
- wall sep 2 
- wall sep 4 

P(H|L first vis)
- first solo
- second solo
- combined solo
- social
- combined solo - social
- wall sep 1
- wall sep 2
- wall sep 4

P(H|opponent vis)
- social
- wall sep 1
- wall sep 2
- wall sep 4

P(H|opponent not vis)
- social
- wall sep 1
- wall sep 2
- wall sep 4

P(win)
- social
- wall sep 1
- wall sep 2
- wall sep 4

Score per unit time
- first solo
- second solo
- combined solo
- social
- wall sep 1
- wall sep 2
- wall sep 4

- match to pseudonym and store in table


SOCIALS

In [83]:
pseudo_tuples_list

[['CY17', 'RL17'],
 ['FA12', 'SL12'],
 ['TE10', 'TK10'],
 ['AM09', 'MC09'],
 ['AL01', 'NL01'],
 ['HL19', 'AW19'],
 ['YL15', 'RR15'],
 ['SS22', 'AD22'],
 ['NK13', 'RD13'],
 ['AV19', 'XG19'],
 ['SH27', 'EN27'],
 ['SP17', 'AW17'],
 ['AR10', 'MY10'],
 ['JU19', 'SY19'],
 ['NO12', 'AS12'],
 ['IN04', 'JL04'],
 ['ZS20', 'VC20'],
 ['KC14', 'WL14'],
 ['MP04', 'SH04'],
 ['PO03', 'NN03'],
 ['HM18', 'SC18'],
 ['NM21', 'KC21'],
 ['EH21', 'SP21'],
 ['JK18', 'HL18'],
 ['JW04', 'JI04'],
 ['KA12', 'WM12'],
 ['ZZ30', 'VW30'],
 ['RK10', 'RU10'],
 ['IN11', 'CW11'],
 ['JP24', 'IP24'],
 ['IS01', 'HQ01'],
 ['YW13', 'JL13'],
 ['YY15', 'MR15'],
 ['PT19', 'JL19'],
 ['CY20', 'MJ20'],
 ['YL13', 'HC13'],
 ['SB19', 'HH19'],
 ['TG12', 'SC12'],
 ['DB04', 'GG04'],
 ['ZH17', 'EM17'],
 ['JC02', 'KS02'],
 ['EX03', 'BC03'],
 ['JS20', 'RR20'],
 ['AE14', 'JL14'],
 ['SH02', 'ML02'],
 ['JW23', 'YL23'],
 ['SJ18', 'CD18'],
 ['KS20', 'CS20'],
 ['MA14', 'RM14'],
 ['LM21', 'LS21']]

In [82]:
trial_list = social_lists[1]

# filter trial list for HighLow trialtype
trial_indices = get_indices.get_trials_trialtype(trial_list, trial_type=globals.HIGH_LOW)
trial_list_filtered = [trial_list[i] for i in trial_indices]
player_id = 0
# get Other visibility for this trial list
orientation_angle_to_other_session = opponent_visibility.get_angle_of_opponent_from_player_session(player_id, trial_list_filtered)
other_visible_session = opponent_visibility.get_other_visible_session(orientation_angle_to_other_session, 110)

test = opponent_visibility.probability_win_trial_conditioned_on_other_visibility(trial_list_filtered, other_visible_session,
                                                                 player_id,
                                                                 inverse_other_visible=False, debug=True)

Next session, player_id 0
Len other_visible_trial_indices: 37
Len trial_list_filtered_this_player_wins: 29
Len other_visible_trial_indices: 37
Probability player wins given these conditions: 0.7837837837837838


In [84]:
for i in range(0,len(social_lists), 2):
    print(i)

0
2
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
40
42
44
46
48


In [116]:
e = np.full(len(social_lists), np.nan)
counts_e = np.full(len(social_lists), np.nan)
f = np.full(len(social_lists), np.nan)
counts_f = np.full(len(social_lists), np.nan)
g = np.full(len(social_lists), np.nan)
counts_g = np.full(len(social_lists), np.nan)
h = np.full(len(social_lists), np.nan)
counts_h = np.full(len(social_lists), np.nan)
trial_list_index = 0
inferred_choice=False
debug=True
current_fov = 110

for i in range(0,len(social_lists)):
    print(trial_list_index)
    trial_list = social_lists[trial_list_index]

    # filter trial list for HighLow trialtype
    trial_indices = get_indices.get_trials_trialtype(trial_list, trial_type=globals.HIGH_LOW)
    trial_list_filtered = [trial_list[i] for i in trial_indices]

    player_id = 0
    # get Other visibility for this trial list
    orientation_angle_to_other_session = opponent_visibility.get_angle_of_opponent_from_player_session(player_id, trial_list_filtered)
    other_visible_session = opponent_visibility.get_other_visible_session(orientation_angle_to_other_session, current_fov)
    
    e[i],counts_e[i] = opponent_visibility.probability_win_trial_conditioned_on_other_visibility(trial_list_filtered, other_visible_session,
                                                                 player_id,
                                                                 inverse_other_visible=False, debug=debug)
    f[i],counts_f[i] = opponent_visibility.probability_win_trial_conditioned_on_other_visibility(trial_list_filtered, other_visible_session,
                                                                 player_id,
                                                                 inverse_other_visible=True, debug=debug)
    if not np.isnan(e[i]):
        print(f"NO nan value for trial list {trial_list_index}")

    player_id = 1
    # get Other visibility for this trial list
    orientation_angle_to_other_session = opponent_visibility.get_angle_of_opponent_from_player_session(player_id, trial_list_filtered)
    other_visible_session = opponent_visibility.get_other_visible_session(orientation_angle_to_other_session, current_fov)
    
    g[i],counts_g[i] = opponent_visibility.probability_win_trial_conditioned_on_other_visibility(trial_list_filtered, other_visible_session,
                                                                   player_id,
                                                                   inverse_other_visible=False, debug=debug)
    h[i],counts_h[i] = opponent_visibility.probability_win_trial_conditioned_on_other_visibility(trial_list_filtered, other_visible_session,
                                                                   player_id,
                                                                   inverse_other_visible=True, debug=debug)
    
    trial_list_index += 1
    
    
wall_choice_probabilities_p0 = [e,f]
wall_choice_probabilities_p1 = [g,h]
trial_counts_p0 = [counts_e,counts_f]
trial_counts_p1 = [counts_g,counts_h]

0
Next session, player_id 0
Len other_visible_trial_indices: 36
Len trial_list_filtered_this_player_wins: 14
Len other_visible_trial_indices: 36
Probability player wins given these conditions: 0.3888888888888889
Next session, player_id 0
Len other_visible_trial_indices: 82
Len trial_list_filtered_this_player_wins: 49
Len other_visible_trial_indices: 82
Probability player wins given these conditions: 0.5975609756097561
NO nan value for trial list 0
Next session, player_id 1
Len other_visible_trial_indices: 79
Len trial_list_filtered_this_player_wins: 34
Len other_visible_trial_indices: 79
Probability player wins given these conditions: 0.43037974683544306
Next session, player_id 1
Len other_visible_trial_indices: 39
Len trial_list_filtered_this_player_wins: 21
Len other_visible_trial_indices: 39
Probability player wins given these conditions: 0.5384615384615384
1
Next session, player_id 0
Len other_visible_trial_indices: 37
Len trial_list_filtered_this_player_wins: 29
Len other_visible_

Wall sep: 45°

In [27]:
e = np.full(len(social_lists), np.nan)
counts_e = np.full(len(social_lists), np.nan)
f = np.full(len(social_lists), np.nan)
counts_f = np.full(len(social_lists), np.nan)
g = np.full(len(social_lists), np.nan)
counts_g = np.full(len(social_lists), np.nan)
h = np.full(len(social_lists), np.nan)
counts_h = np.full(len(social_lists), np.nan)
trial_list_index = 0
inferred_choice=False
debug=True
current_fov = 110

for i in range(0,len(social_lists)):

    trial_list = social_lists[trial_list_index]

    # filter trial list for HighLow trialtype
    trial_indices = get_indices.get_trials_trialtype(trial_list, trial_type=globals.HIGH_LOW)
    trial_list_filtered = [trial_list[i] for i in trial_indices]

    #filter trial list for 45° wall separation
    trial_indices = get_indices.get_trials_with_wall_sep(trial_list_filtered, wall_sep=1)
    trial_list_filtered = [trial_list_filtered[i] for i in trial_indices]

    player_id = 0
    # get Other visibility for this trial list
    orientation_angle_to_other_session = opponent_visibility.get_angle_of_opponent_from_player_session(player_id, trial_list_filtered)
    other_visible_session = opponent_visibility.get_other_visible_session(orientation_angle_to_other_session, current_fov)
    
    e[i], counts_e[i] = opponent_visibility.probability_win_trial_conditioned_on_other_visibility(trial_list_filtered, other_visible_session,
                                                                 player_id,
                                                                 inverse_other_visible=False, debug=debug)
    f[i], counts_f[i] = opponent_visibility.probability_win_trial_conditioned_on_other_visibility(trial_list_filtered, other_visible_session,
                                                                 player_id,
                                                                 inverse_other_visible=True, debug=debug)
    
    player_id = 1
    # get Other visibility for this trial list
    orientation_angle_to_other_session = opponent_visibility.get_angle_of_opponent_from_player_session(player_id, trial_list_filtered)
    other_visible_session = opponent_visibility.get_other_visible_session(orientation_angle_to_other_session, current_fov)
    
    g[i], counts_g[i] = opponent_visibility.probability_win_trial_conditioned_on_other_visibility(trial_list_filtered, other_visible_session,
                                                                   player_id,
                                                                   inverse_other_visible=False, debug=debug)
    h[i], counts_h[i] = opponent_visibility.probability_win_trial_conditioned_on_other_visibility(trial_list_filtered, other_visible_session,
                                                                   player_id,
                                                                   inverse_other_visible=True, debug=debug)
    
    trial_list_index += 1
    
    
wall_choice_probabilities1_p0 = [e,f]
wall_choice_probabilities1_p1 = [g,h]
trial_counts1_p0 = [counts_e,counts_f]
trial_counts1_p1 = [counts_g,counts_h]

Next session, player_id 0
Len other_visible_trial_indices: 22
Len trial_list_filtered_this_player_wins: 9
Len other_visible_trial_indices: 22
Probability player wins given these conditions: 0.4090909090909091
Next session, player_id 0
Len other_visible_trial_indices: 41
Len trial_list_filtered_this_player_wins: 28
Len other_visible_trial_indices: 41
Probability player wins given these conditions: 0.6829268292682927
Next session, player_id 1
Len other_visible_trial_indices: 43
Len trial_list_filtered_this_player_wins: 15
Len other_visible_trial_indices: 43
Probability player wins given these conditions: 0.3488372093023256
Next session, player_id 1
Len other_visible_trial_indices: 20
Len trial_list_filtered_this_player_wins: 11
Len other_visible_trial_indices: 20
Probability player wins given these conditions: 0.55
Next session, player_id 0
Len other_visible_trial_indices: 21
Len trial_list_filtered_this_player_wins: 18
Len other_visible_trial_indices: 21
Probability player wins given th

Wall sep: 90°

In [28]:
e = np.full(len(social_lists), np.nan)
counts_e = np.full(len(social_lists), np.nan)
f = np.full(len(social_lists), np.nan)
counts_f = np.full(len(social_lists), np.nan)
g = np.full(len(social_lists), np.nan)
counts_g = np.full(len(social_lists), np.nan)
h = np.full(len(social_lists), np.nan)
counts_h = np.full(len(social_lists), np.nan)
trial_list_index = 0
inferred_choice=False
debug=True
current_fov = 110

for i in range(0,len(social_lists)):

    trial_list = social_lists[trial_list_index]

    # filter trial list for HighLow trialtype
    trial_indices = get_indices.get_trials_trialtype(trial_list, trial_type=globals.HIGH_LOW)
    trial_list_filtered = [trial_list[i] for i in trial_indices]

    #filter trial list for 45° wall separation
    trial_indices = get_indices.get_trials_with_wall_sep(trial_list_filtered, wall_sep=2)
    trial_list_filtered = [trial_list_filtered[i] for i in trial_indices]

    player_id = 0
    # get Other visibility for this trial list
    orientation_angle_to_other_session = opponent_visibility.get_angle_of_opponent_from_player_session(player_id, trial_list_filtered)
    other_visible_session = opponent_visibility.get_other_visible_session(orientation_angle_to_other_session, current_fov)
    
    e[i],counts_e[i] = opponent_visibility.probability_win_trial_conditioned_on_other_visibility(trial_list_filtered, other_visible_session,
                                                                 player_id,
                                                                 inverse_other_visible=False, debug=debug)
    f[i],counts_f[i] = opponent_visibility.probability_win_trial_conditioned_on_other_visibility(trial_list_filtered, other_visible_session,
                                                                 player_id,
                                                                 inverse_other_visible=True, debug=debug)
    
    player_id = 1
    # get Other visibility for this trial list
    orientation_angle_to_other_session = opponent_visibility.get_angle_of_opponent_from_player_session(player_id, trial_list_filtered)
    other_visible_session = opponent_visibility.get_other_visible_session(orientation_angle_to_other_session, current_fov)
    
    g[i],counts_g[i] = opponent_visibility.probability_win_trial_conditioned_on_other_visibility(trial_list_filtered, other_visible_session,
                                                                   player_id,
                                                                   inverse_other_visible=False, debug=debug)
    h[i],counts_h[i] = opponent_visibility.probability_win_trial_conditioned_on_other_visibility(trial_list_filtered, other_visible_session,
                                                                   player_id,
                                                                   inverse_other_visible=True, debug=debug)
    
    trial_list_index += 1
    
    
wall_choice_probabilities2_p0 = [e,f]
wall_choice_probabilities2_p1 = [g,h]
trial_counts2_p0 = [counts_e,counts_f]
trial_counts2_p1 = [counts_g,counts_h]

Next session, player_id 0
Len other_visible_trial_indices: 9
Len trial_list_filtered_this_player_wins: 4
Len other_visible_trial_indices: 9
Probability player wins given these conditions: 0.4444444444444444
Next session, player_id 0
Len other_visible_trial_indices: 19
Len trial_list_filtered_this_player_wins: 8
Len other_visible_trial_indices: 19
Probability player wins given these conditions: 0.42105263157894735
Next session, player_id 1
Len other_visible_trial_indices: 17
Len trial_list_filtered_this_player_wins: 9
Len other_visible_trial_indices: 17
Probability player wins given these conditions: 0.5294117647058824
Next session, player_id 1
Len other_visible_trial_indices: 11
Len trial_list_filtered_this_player_wins: 7
Len other_visible_trial_indices: 11
Probability player wins given these conditions: 0.6363636363636364
Next session, player_id 0
Len other_visible_trial_indices: 8
Len trial_list_filtered_this_player_wins: 6
Len other_visible_trial_indices: 8
Probability player wins g

Wall sep: 180°

In [29]:
e = np.full(len(social_lists), np.nan)
counts_e = np.full(len(social_lists), np.nan)
f = np.full(len(social_lists), np.nan)
counts_f = np.full(len(social_lists), np.nan)
g = np.full(len(social_lists), np.nan)
counts_g = np.full(len(social_lists), np.nan)
h = np.full(len(social_lists), np.nan)
counts_h = np.full(len(social_lists), np.nan)
trial_list_index = 0
inferred_choice=False
debug=True
current_fov = 110

for i in range(0,len(social_lists)):

    trial_list = social_lists[trial_list_index]

    # filter trial list for HighLow trialtype
    trial_indices = get_indices.get_trials_trialtype(trial_list, trial_type=globals.HIGH_LOW)
    trial_list_filtered = [trial_list[i] for i in trial_indices]

    #filter trial list for 45° wall separation
    trial_indices = get_indices.get_trials_with_wall_sep(trial_list_filtered, wall_sep=4)
    trial_list_filtered = [trial_list_filtered[i] for i in trial_indices]

    player_id = 0
    # get Other visibility for this trial list
    orientation_angle_to_other_session = opponent_visibility.get_angle_of_opponent_from_player_session(player_id, trial_list_filtered)
    other_visible_session = opponent_visibility.get_other_visible_session(orientation_angle_to_other_session, current_fov)
    
    e[i],counts_e[i] = opponent_visibility.probability_win_trial_conditioned_on_other_visibility(trial_list_filtered, other_visible_session,
                                                                 player_id,
                                                                 inverse_other_visible=False, debug=debug)
    f[i],counts_f[i] = opponent_visibility.probability_win_trial_conditioned_on_other_visibility(trial_list_filtered, other_visible_session,
                                                                 player_id,
                                                                 inverse_other_visible=True, debug=debug)
    
    player_id = 1
    # get Other visibility for this trial list
    orientation_angle_to_other_session = opponent_visibility.get_angle_of_opponent_from_player_session(player_id, trial_list_filtered)
    other_visible_session = opponent_visibility.get_other_visible_session(orientation_angle_to_other_session, current_fov)
    
    g[i],counts_g[i] = opponent_visibility.probability_win_trial_conditioned_on_other_visibility(trial_list_filtered, other_visible_session,
                                                                   player_id,
                                                                   inverse_other_visible=False, debug=debug)
    h[i],counts_h[i] = opponent_visibility.probability_win_trial_conditioned_on_other_visibility(trial_list_filtered, other_visible_session,
                                                                   player_id,
                                                                   inverse_other_visible=True, debug=debug)
    
    trial_list_index += 1
    
    
wall_choice_probabilities4_p0 = [e,f]
wall_choice_probabilities4_p1 = [g,h]
trial_counts4_p0 = [counts_e,counts_f]
trial_counts4_p1 = [counts_g,counts_h]

Next session, player_id 0
Len other_visible_trial_indices: 5
Len trial_list_filtered_this_player_wins: 1
Len other_visible_trial_indices: 5
Probability player wins given these conditions: 0.2
Next session, player_id 0
Len other_visible_trial_indices: 22
Len trial_list_filtered_this_player_wins: 13
Len other_visible_trial_indices: 22
Probability player wins given these conditions: 0.5909090909090909
Next session, player_id 1
Len other_visible_trial_indices: 19
Len trial_list_filtered_this_player_wins: 10
Len other_visible_trial_indices: 19
Probability player wins given these conditions: 0.5263157894736842
Next session, player_id 1
Len other_visible_trial_indices: 8
Len trial_list_filtered_this_player_wins: 3
Len other_visible_trial_indices: 8
Probability player wins given these conditions: 0.375
Next session, player_id 0
Len other_visible_trial_indices: 8
Len trial_list_filtered_this_player_wins: 5
Len other_visible_trial_indices: 8
Probability player wins given these conditions: 0.625


Pseudonym assignments + storing in dictionaries

In [117]:
p_win_other_visible_sep_1 = {}
p_win_other_visible_sep_2 = {}
p_win_other_visible_sep_4 = {}
p_win_other_visible = {}

p_win_other_not_visible_sep_1 = {}
p_win_other_not_visible_sep_2 = {}
p_win_other_not_visible_sep_4 = {}
p_win_other_not_visible = {}

for suffix in ['probability', 'denominator']:
    p_win_other_visible_sep_1[suffix] = {}
    p_win_other_visible_sep_2[suffix] = {}
    p_win_other_visible_sep_4[suffix] = {}
    p_win_other_visible[suffix] = {}

    p_win_other_not_visible_sep_1[suffix] = {}
    p_win_other_not_visible_sep_2[suffix] = {}
    p_win_other_not_visible_sep_4[suffix] = {}
    p_win_other_not_visible[suffix] = {}

In [119]:
trial_counts_p0

[array([ 36.,  37.,  30.,  47., 111.,  68.,  37., 103.,  24.,  24.,   7.,
         37.,  44.,  32.,  28.,  26.,  40.,  31.,  30.,  37.,  31.,  27.,
        110.,  33.,  51.,  54.,  31.,  32.,  45.,  49.,  38.,  44.,  32.,
         77.,  34.,  37.,  33.,  69.,  33.,  44.,  45.,  30.,  56.,  35.,
         26.,  32.,  36.,  96.,  50.,  48.]),
 array([82., 56., 80., 57.,  4., 46., 72., 25., 69., 69., 72., 56., 68.,
        82., 70., 15., 79., 52., 82., 50., 62., 83.,  9., 71., 59., 23.,
        94., 83., 76., 68., 77., 58., 71., 33., 73., 72., 65., 43., 78.,
        92., 76., 65., 68., 64., 89., 66., 70.,  4., 57., 64.])]

In [114]:
for i, tuple in enumerate(pseudo_tuples_list):
    player0 = tuple[0]
    player1 = tuple[1]
    print(player1)
    print(trial_counts_p1[0][i])

RL17
nan
SL12
nan
TK10
nan
MC09
nan
NL01
nan
AW19
nan
RR15
nan
AD22
nan
RD13
nan
XG19
nan
EN27
nan
AW17
nan
MY10
nan
SY19
nan
AS12
nan
JL04
nan
VC20
nan
WL14
nan
SH04
nan
NN03
nan
SC18
nan
KC21
nan
SP21
nan
HL18
nan
JI04
nan
WM12
nan
VW30
nan
RU10
nan
CW11
nan
IP24
nan
HQ01
nan
JL13
nan
MR15
nan
JL19
nan
MJ20
nan
HC13
nan
HH19
nan
SC12
nan
GG04
nan
EM17
nan
KS02
nan
BC03
nan
RR20
nan
JL14
nan
ML02
nan
YL23
nan
CD18
nan
CS20
nan
RM14
nan
LS21
nan


In [120]:
#socials
for suffix in ['probability', 'denominator']:
    for i, tuple in enumerate(pseudo_tuples_list):
        player0 = tuple[0]
        player1 = tuple[1]
        
        if player0 not in p_win_other_visible:
            p_win_other_visible_sep_1[suffix][player0] = 0
            p_win_other_visible_sep_2[suffix][player0] = 0
            p_win_other_visible_sep_4[suffix][player0] = 0
            p_win_other_visible[suffix][player0] = 0

            p_win_other_not_visible_sep_1[suffix][player0] = 0
            p_win_other_not_visible_sep_2[suffix][player0] = 0
            p_win_other_not_visible_sep_4[suffix][player0] = 0
            p_win_other_not_visible[suffix][player0] = 0
        
        if player1 not in p_win_other_visible:
            p_win_other_visible_sep_1[suffix][player1] = 0
            p_win_other_visible_sep_2[suffix][player1] = 0
            p_win_other_visible_sep_4[suffix][player1] = 0
            p_win_other_visible[suffix][player1] = 0

            p_win_other_not_visible_sep_1[suffix][player1] = 0
            p_win_other_not_visible_sep_2[suffix][player1] = 0
            p_win_other_not_visible_sep_4[suffix][player1] = 0
            p_win_other_not_visible[suffix][player1] = 0


for i, tuple in enumerate(pseudo_tuples_list):
    player0 = tuple[0]
    player1 = tuple[1]

    print(player1)
    
    p_win_other_visible_sep_1['probability'][player0] = wall_choice_probabilities1_p0[0][i]
    p_win_other_visible_sep_2['probability'][player0] = wall_choice_probabilities2_p0[0][i]
    p_win_other_visible_sep_4['probability'][player0] = wall_choice_probabilities4_p0[0][i]
    p_win_other_visible['probability'][player0] = wall_choice_probabilities_p0[0][i]

    p_win_other_not_visible_sep_1['probability'][player0] = wall_choice_probabilities1_p0[1][i]
    p_win_other_not_visible_sep_2['probability'][player0] = wall_choice_probabilities2_p0[1][i]
    p_win_other_not_visible_sep_4['probability'][player0] = wall_choice_probabilities4_p0[1][i]
    p_win_other_not_visible['probability'][player0] = wall_choice_probabilities_p0[1][i]

    p_win_other_visible_sep_1['probability'][player1] = wall_choice_probabilities1_p1[0][i]
    p_win_other_visible_sep_2['probability'][player1] = wall_choice_probabilities2_p1[0][i]
    p_win_other_visible_sep_4['probability'][player1] = wall_choice_probabilities4_p1[0][i]
    p_win_other_visible['probability'][player1] = wall_choice_probabilities_p1[0][i]

    p_win_other_not_visible_sep_1['probability'][player1] = wall_choice_probabilities1_p1[1][i]
    p_win_other_not_visible_sep_2['probability'][player1] = wall_choice_probabilities2_p1[1][i]
    p_win_other_not_visible_sep_4['probability'][player1] = wall_choice_probabilities4_p1[1][i]
    p_win_other_not_visible['probability'][player1] = wall_choice_probabilities_p1[1][i]

    p_win_other_visible_sep_1['denominator'][player0] = trial_counts1_p0[0][i]
    p_win_other_visible_sep_2['denominator'][player0] = trial_counts2_p0[0][i]
    p_win_other_visible_sep_4['denominator'][player0] = trial_counts4_p0[0][i]
    p_win_other_visible['denominator'][player0] = trial_counts_p0[0][i]

    p_win_other_not_visible_sep_1['denominator'][player0] = trial_counts1_p0[1][i]
    p_win_other_not_visible_sep_2['denominator'][player0] = trial_counts2_p0[1][i]
    p_win_other_not_visible_sep_4['denominator'][player0] = trial_counts4_p0[1][i]
    p_win_other_not_visible['denominator'][player0] = trial_counts_p0[1][i]

    p_win_other_visible_sep_1['denominator'][player1] = trial_counts1_p1[0][i]
    p_win_other_visible_sep_2['denominator'][player1] = trial_counts2_p1[0][i]
    p_win_other_visible_sep_4['denominator'][player1] = trial_counts4_p1[0][i]
    p_win_other_visible['denominator'][player1] = trial_counts_p1[0][i]

    p_win_other_not_visible_sep_1['denominator'][player1] = trial_counts1_p1[1][i]
    p_win_other_not_visible_sep_2['denominator'][player1] = trial_counts2_p1[1][i]
    p_win_other_not_visible_sep_4['denominator'][player1] = trial_counts4_p1[1][i]
    p_win_other_not_visible['denominator'][player1] = trial_counts_p1[1][i]


RL17
SL12
TK10
MC09
NL01
AW19
RR15
AD22
RD13
XG19
EN27
AW17
MY10
SY19
AS12
JL04
VC20
WL14
SH04
NN03
SC18
KC21
SP21
HL18
JI04
WM12
VW30
RU10
CW11
IP24
HQ01
JL13
MR15
JL19
MJ20
HC13
HH19
SC12
GG04
EM17
KS02
BC03
RR20
JL14
ML02
YL23
CD18
CS20
RM14
LS21


In [122]:
df = pd.read_csv('individual_analyses_table_5.csv', index_col=0)

In [123]:
for pseudonym in df.index:
    print(pseudonym)
    print(p_win_other_visible['probability'][pseudonym])
    df.loc[pseudonym, 'p_win_other_visible_sep_1'] = p_win_other_visible_sep_1['probability'][pseudonym]
    df.loc[pseudonym, 'p_win_other_visible_sep_2'] = p_win_other_visible_sep_2['probability'][pseudonym]
    df.loc[pseudonym, 'p_win_other_visible_sep_4'] = p_win_other_visible_sep_4['probability'][pseudonym]
    df.loc[pseudonym, 'p_win_other_visible'] = p_win_other_visible['probability'][pseudonym]
    
    df.loc[pseudonym, 'p_win_other_not_visible_sep_1'] = p_win_other_not_visible_sep_1['probability'][pseudonym]
    df.loc[pseudonym, 'p_win_other_not_visible_sep_2'] = p_win_other_not_visible_sep_2['probability'][pseudonym]
    df.loc[pseudonym, 'p_win_other_not_visible_sep_4'] = p_win_other_not_visible_sep_4['probability'][pseudonym]
    df.loc[pseudonym, 'p_win_other_not_visible'] = p_win_other_not_visible['probability'][pseudonym]

SP17
0.24324324324324326
VC20
0.675
IN04
0.6923076923076923
JL19
0.2857142857142857
BC03
0.21428571428571427
NM21
0.3333333333333333
XG19
0.4642857142857143
MY10
0.2222222222222222
YW13
0.4090909090909091
MA14
0.36
IS01
0.3157894736842105
MJ20
0.2976190476190476
YL13
0.2972972972972973
EH21
0.509090909090909
IN11
0.35555555555555557
SH02
0.34615384615384615
JW23
0.59375
KA12
0.37037037037037035
KC14
0.06451612903225806
RD13
0.5185185185185185
RR15
0.7684210526315789
SL12
0.16279069767441862
HH19
0.15555555555555556
YL23
0.3611111111111111
RR20
0.7307692307692307
JL04
0.15384615384615385
AV19
0.3333333333333333
NO12
0.32142857142857145
AD22
0.5555555555555556
EM17
0.6595744680851063
EX03
0.43333333333333335
NK13
0.5
RL17
0.43037974683544306
JP24
0.3877551020408163
SC12
0.6296296296296297
WM12
0.5909090909090909
JW04
0.5294117647058824
TG12
0.4057971014492754
SH04
0.23809523809523808
CY20
0.6764705882352942
RK10
0.3125
HL19
0.4411764705882353
JL14
0.5783132530120482
SJ18
0.36111111111111

In [124]:
df

,experiment_date,pH_H_first_vis_combined_solo_sep1,pH_H_first_vis_combined_solo_sep2,pH_H_first_vis_combined_solo_sep4,pH_H_first_vis_combined_solo_all_seps,pH_H_first_vis_first_solo_all_seps,pH_H_first_vis_second_solo_all_seps,pH_H_first_vis_social_sep1,pH_H_first_vis_social_sep2,pH_H_first_vis_social_sep4,pH_H_first_vis_social_all_seps,pH_L_first_vis_combined_solo_sep1,pH_L_first_vis_combined_solo_sep2,pH_L_first_vis_combined_solo_sep4,pH_L_first_vis_combined_solo_all_seps,pH_L_first_vis_first_solo_all_seps,pH_L_first_vis_second_solo_all_seps,pH_L_first_vis_social_sep1,pH_L_first_vis_social_sep2,pH_L_first_vis_social_sep4,pH_L_first_vis_social_all_seps,sensory_effect_solo_sep1,sensory_effect_social_sep1,solo-social_sensory_effect_sep1,sensory_effect_solo_sep2,sensory_effect_social_sep2,solo-social_sensory_effect_sep2,sensory_effect_solo_sep4,sensory_effect_social_sep4,solo-social_sensory_effect_sep4,sensory_effect_solo_all_seps,sensory_effect_social_all_seps,solo-social_sensory_effect_all_seps,pH_opp_visible_sep_1,pH_opp_visible_sep_2,pH_opp_visible_sep_4,pH_opp_visible_all_seps,pH_opp_not_visible_sep_1,pH_opp_not_visible_sep_2,pH_opp_not_visible_sep_4,pH_opp_not_visible_all_seps,p_win_other_visible_sep_1,p_win_other_visible_sep_2,p_win_other_visible_sep_4,p_win_other_visible,p_win_other_not_visible_sep_1,p_win_other_not_visible_sep_2,p_win_other_not_visible_sep_4,p_win_other_not_visible
SP17,2024-10-17,NaN,NaN,NaN,0.923076923,NaN,1.000000000,0.700000000,0.700000000,0.857142857,0.740740741,NaN,1.000000000,0.285714286,0.714285714,0.714285714,0.714285714,0.444444444,NaN,0.111111111,0.291666667,NaN,0.255555556,NaN,NaN,NaN,NaN,NaN,0.746031746,NaN,0.208791209,0.449074074,-0.240282865,0.833333333,0.444444444,0.375000000,0.586206897,0.685714286,0.545454545,0.444444444,0.586666667,0.428571429,0.181818182,0.083333333,0.243243243,0.518518519,0.437500000,0.538461538,0.500000000
VC20,2024-11-20,0.909090909,1.000000000,1.000000000,0.961538462,0.900000000,1.000000000,0.928571429,0.866666667,0.812500000,0.866666667,1.000000000,NaN,NaN,1.000000000,1.000000000,1.000000000,0.937500000,0.375000000,0.235294118,0.536585366,-0.090909091,-0.008928571,-0.081980519,NaN,0.491666667,NaN,NaN,0.577205882,NaN,-0.038461538,0.330081301,-0.368542839,1.000000000,0.750000000,0.555555556,0.825000000,0.941176471,0.696969697,0.515151515,0.752136752,0.789473684,0.666666667,0.444444444,0.675000000,0.606060606,0.714285714,0.640000000,0.645569620
IN04,2025-02-04,0.909090909,NaN,1.000000000,0.947368421,0.909090909,1.000000000,1.000000000,NaN,NaN,1.000000000,1.000000000,1.000000000,1.000000000,1.000000000,1.000000000,1.000000000,1.000000000,NaN,NaN,0.769230769,-0.090909091,0.000000000,-0.090909091,NaN,NaN,NaN,0.000000000,NaN,NaN,-0.052631579,0.230769231,-0.283400810,1.000000000,NaN,NaN,0.923076923,1.000000000,0.875000000,0.714285714,0.923076923,0.733333333,0.800000000,0.500000000,0.692307692,0.900000000,1.000000000,0.500000000,0.866666667
JL19,2025-03-19,1.000000000,NaN,1.000000000,1.000000000,1.000000000,NaN,0.916666667,0.714285714,0.200000000,0.620689655,NaN,0.875000000,1.000000000,0.952380952,0.916666667,1.000000000,0.384615385,0.375000000,0.250000000,0.344827586,NaN,0.532051282,NaN,NaN,0.339285714,NaN,0.000000000,-0.050000000,0.050000000,0.047619048,0.275862069,-0.228243021,0.444444444,0.411764706,0.142857143,0.392156863,0.583333333,0.458333333,0.285714286,0.483870968,0.218750000,0.380952381,0.300000000,0.285714286,0.346153846,0.571428571,0.428571429,0.404255319
BC03,2024-12-03,1.000000000,NaN,NaN,1.000000000,NaN,1.000000000,1.000000000,0.714285714,0.933333333,0.896551724,NaN,0.428571429,0.857142857,0.666666667,NaN,0.750000000,NaN,NaN,0.000000000,0.000000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.933333333,NaN,0.333333333,0.896551724,-0.563218391,NaN,NaN,NaN,0.400000000,0.551724138,0.437500000,0.500000000,0.506666667,0.250000000,0.000000000,0.250000000,0.214285714,0.405405405,0.411764706,0.703703704,0.506172840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [125]:
df.to_csv('individual_analyses_table_6.csv')

In [126]:
denom_df = pd.read_csv('individual_analyses_trial_counts_5.csv', index_col=0)

In [101]:
p_win_other_visible['denominator']

{'CY17': np.float64(36.0),
 'RL17': np.float64(nan),
 'FA12': np.float64(37.0),
 'SL12': np.float64(nan),
 'TE10': np.float64(30.0),
 'TK10': np.float64(nan),
 'AM09': np.float64(47.0),
 'MC09': np.float64(nan),
 'AL01': np.float64(111.0),
 'NL01': np.float64(nan),
 'HL19': np.float64(68.0),
 'AW19': np.float64(nan),
 'YL15': np.float64(37.0),
 'RR15': np.float64(nan),
 'SS22': np.float64(103.0),
 'AD22': np.float64(nan),
 'NK13': np.float64(24.0),
 'RD13': np.float64(nan),
 'AV19': np.float64(24.0),
 'XG19': np.float64(nan),
 'SH27': np.float64(7.0),
 'EN27': np.float64(nan),
 'SP17': np.float64(37.0),
 'AW17': np.float64(nan),
 'AR10': np.float64(44.0),
 'MY10': np.float64(nan),
 'JU19': np.float64(32.0),
 'SY19': np.float64(nan),
 'NO12': np.float64(28.0),
 'AS12': np.float64(nan),
 'IN04': np.float64(26.0),
 'JL04': np.float64(nan),
 'ZS20': np.float64(40.0),
 'VC20': np.float64(nan),
 'KC14': np.float64(31.0),
 'WL14': np.float64(nan),
 'MP04': np.float64(30.0),
 'SH04': np.float6

In [127]:
for pseudonym in denom_df.index:
    denom_df.loc[pseudonym, 'p_win_other_visible_sep_1'] = p_win_other_visible_sep_1['denominator'][pseudonym]
    denom_df.loc[pseudonym, 'p_win_other_visible_sep_2'] = p_win_other_visible_sep_2['denominator'][pseudonym]
    denom_df.loc[pseudonym, 'p_win_other_visible_sep_4'] = p_win_other_visible_sep_4['denominator'][pseudonym]
    denom_df.loc[pseudonym, 'p_win_other_visible'] = p_win_other_visible['denominator'][pseudonym]
    
    denom_df.loc[pseudonym, 'p_win_other_not_visible_sep_1'] = p_win_other_not_visible_sep_1['denominator'][pseudonym]
    denom_df.loc[pseudonym, 'p_win_other_not_visible_sep_2'] = p_win_other_not_visible_sep_2['denominator'][pseudonym]
    denom_df.loc[pseudonym, 'p_win_other_not_visible_sep_4'] = p_win_other_not_visible_sep_4['denominator'][pseudonym]
    denom_df.loc[pseudonym, 'p_win_other_not_visible'] = p_win_other_not_visible['denominator'][pseudonym]

In [130]:
denom_df

,experiment_date,pH_H_first_combined_solo_sep1,pH_H_first_combined_solo_sep2,pH_H_first_combined_solo_sep4,pH_H_first_combined_solo_all_seps,pH_H_first_first_solo_all_seps,pH_H_first_second_solo_all_seps,pH_H_first_social_sep1,pH_H_first_social_sep2,pH_H_first_social_sep4,pH_H_first_social_all_seps,pH_L_first_vis_combined_solo_sep1,pH_L_first_vis_combined_solo_sep2,pH_L_first_vis_combined_solo_sep4,pH_L_first_vis_combined_solo_all_seps,pH_L_first_vis_first_solo_all_seps,pH_L_first_vis_second_solo_all_seps,pH_L_first_vis_social_sep1,pH_L_first_vis_social_sep2,pH_L_first_vis_social_sep4,pH_L_first_vis_social_all_seps,pH_opp_visible_sep_1,pH_opp_visible_sep_2,pH_opp_visible_sep_4,pH_opp_visible_all_seps,pH_opp_not_visible_sep_1,pH_opp_not_visible_sep_2,pH_opp_not_visible_sep_4,pH_opp_not_visible_all_seps,rt_by_traj_efficiency_first_solo,rt_by_traj_efficiency_second_solo,rt_by_traj_efficiency_combined_solo,rt_by_traj_efficiency_wins,rt_by_traj_efficiency_losses,p_win_other_visible_sep_1,p_win_other_visible_sep_2,p_win_other_visible_sep_4,p_win_other_visible,p_win_other_not_visible_sep_1,p_win_other_not_visible_sep_2,p_win_other_not_visible_sep_4,p_win_other_not_visible
SP17,2024-10-17,4.0,6.0,3.0,13.0,6.0,7.0,10,10,7,27,6.0,8.0,7.0,21.0,7.0,14.0,9.0,6.0,9.0,24.0,12.0,9.0,8.0,29.0,35.0,22.0,18.0,75.0,22.0,35.0,57.0,39.0,33.0,14.0,11.0,12.0,37.0,27.0,16.0,13.0,56.0
VC20,2024-11-20,11.0,7.0,8.0,26.0,10.0,16.0,14,15,16,45,9.0,4.0,2.0,15.0,8.0,7.0,16.0,8.0,17.0,41.0,19.0,12.0,9.0,40.0,51.0,33.0,33.0,117.0,24.0,32.0,56.0,95.0,42.0,19.0,12.0,9.0,40.0,33.0,21.0,25.0,79.0
IN04,2025-02-04,11.0,1.0,7.0,19.0,11.0,8.0,8,5,4,17,12.0,10.0,8.0,30.0,17.0,13.0,7.0,3.0,3.0,13.0,15.0,5.0,6.0,26.0,24.0,8.0,7.0,39.0,48.0,35.0,83.0,43.0,11.0,15.0,5.0,6.0,26.0,10.0,3.0,2.0,15.0
JL19,2025-03-19,7.0,5.0,7.0,19.0,15.0,4.0,12,7,10,29,6.0,8.0,7.0,21.0,12.0,9.0,13.0,8.0,8.0,29.0,27.0,17.0,7.0,51.0,48.0,24.0,21.0,93.0,56.0,51.0,107.0,47.0,62.0,32.0,21.0,10.0,63.0,26.0,7.0,14.0,47.0
BC03,2024-12-03,11.0,6.0,1.0,18.0,6.0,12.0,7,7,15,29,4.0,7.0,7.0,18.0,6.0,12.0,6.0,4.0,13.0,23.0,6.0,1.0,3.0,10.0,29.0,16.0,30.0,75.0,23.0,28.0,51.0,56.0,17.0,8.0,2.0,4.0,14.0,37.0,17.0,27.0,81.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LS21,2025-04-21,9.0,5.0,10.0,24.0,13.0,11.0,11,4,7,22,7.0,2.0,8.0,17.0,5.0,12.0,11.0,4.0,12.0,27.0,39.0,17.0,28.0,84.0,52.0,20.0,35.0,107.0,54.0,53.0,107.0,78.0,37.0,41.0,17.0,29.0,87.0,15.0,3.0,7.0,25.0
NN03,2024-12-03,12.0,4.0,7.0,23.0,14.0,9.0,14,9,5,28,9.0,1.0,6.0,16.0,8.0,8.0,12.0,5.0,16.0,33.0,14.0,2.0,3.0,19.0,36.0,16.0,21.0,73.0,36.0,26.0,62.0,30.0,39.0,19.0,3.0,3.0,25.0,27.0,16.0,19.0,62.0
KS20,2024-12-20,6.0,13.0,6.0,25.0,13.0,12.0,7,16,6,29,1.0,6.0,2.0,9.0,5.0,4.0,10.0,16.0,4.0,30.0,23.0,44.0,13.0,80.0,23.0,46.0,14.0,83.0,44.0,36.0,80.0,79.0,11.0,25.0,58.0,13.0,96.0,0.0,3.0,1.0,4.0
HQ01,2025-04-01,11.0,9.0,7.0,27.0,13.0,14.0,6,4,3,13,11.0,6.0,9.0,26.0,14.0,12.0,7.0,7.0,6.0,20.0,43.0,26.0,34.0,103.0,46.0,27.0,34.0,107.0,62.0,61.0,123.0,73.0,45.0,45.0,28.0,36.0,109.0,3.0,2.0,1.0,6.0


In [131]:
denom_df.to_csv('individual_analyses_trial_counts_6.csv')